In [43]:
import pandas as pd
import joblib

In [44]:
df = pd.read_json("DataSet_Entrenamiento_v1.json")

In [45]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,5386-THSLQ,Female,1,Yes,No,66,No,No phone service,DSL,No,...,Yes,No,Yes,No,One year,No,Bank transfer (automatic),45.55,3027.25,No
4,3192-NQECA,Male,0,Yes,No,68,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),110.00,7611.85,Yes


In [46]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [47]:
df.drop("customerID",axis = 1, inplace=True)

In [48]:
df.TotalCharges = pd.to_numeric(df.TotalCharges)

In [49]:
num_cols = df._get_numeric_data().columns

In [50]:
cat_cols = df.select_dtypes(include=['object']).columns

In [51]:
for col in cat_cols:
  print(col,df[col].unique())

gender ['Female' 'Male']
Partner ['Yes' 'No']
Dependents ['No' 'Yes']
PhoneService ['No' 'Yes']
MultipleLines ['No phone service' 'No' 'Yes']
InternetService ['DSL' 'Fiber optic' 'No']
OnlineSecurity ['No' 'Yes' 'No internet service']
OnlineBackup ['Yes' 'No' 'No internet service']
DeviceProtection ['No' 'Yes' 'No internet service']
TechSupport ['No' 'Yes' 'No internet service']
StreamingTV ['No' 'Yes' 'No internet service']
StreamingMovies ['No' 'Yes' 'No internet service']
Contract ['Month-to-month' 'One year' 'Two year']
PaperlessBilling ['Yes' 'No']
PaymentMethod ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn ['No' 'Yes']


In [52]:
one = ["gender","Partner","Dependents","PhoneService","PaperlessBilling"]
label = ["MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport",
         "StreamingTV","StreamingMovies","Contract","PaymentMethod"]

Vemos que las variables con dos valores las podemos codificar con un label encoder y el resto con one-hot encoder

In [53]:
from sklearn.model_selection import train_test_split

y = df.Churn
X = df.drop("Churn",axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [54]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant'))
      ,('encoder', OrdinalEncoder())
])

categorical_transformer2 = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant'))
      ,('encoder', OneHotEncoder())
])

In [55]:


preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, num_cols)
   ,('categorical', categorical_transformer, label)
   ,('categorical2',categorical_transformer2,one)
]) 

In [56]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('classifier',RandomForestClassifier())
           ])

In [57]:
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['Multiple...
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaymentMethod']),
                                                 ('categorical2',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'PaperlessBilling'])])),
                ('classifier', RandomForestClassifier())])

In [58]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'classifier__n_estimators': n_estimators,
               'classifier__max_features': max_features,
               'classifier__max_depth': max_depth,
               'classifier__min_samples_split': min_samples_split,
               'classifier__min_samples_leaf': min_samples_leaf,
               'classifier__bootstrap': bootstrap}

In [59]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(pipeline, random_grid, cv=10, verbose=1,n_jobs=-1,scoring='roc_auc')
rs.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('numeric',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                                              ('categorical',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='constant')),
                                                                                               ('enco...
                   param_distributions={'classifier__bootstrap': [True, False],
                                        'classifier__max_depth': [10, 20, 30,
                                                                  40, 50, 60,
                                                                  70, 80, 90,
                                                                  100, 110,
                                                                  None],
                                        'classifier__max_features': ['auto',
                                                                     'sqrt'],
                                        'classifier__min_samples_leaf': [1, 2,
                                                                         4],
                                        'classifier__min_samples_split': [2, 5,
                                                                          10],
                                        'classifier__n_estimators': [200, 400,
                                                                     600, 800,
                                                                     1000, 1200,
                                                                     1400, 1600,
                                                                     1800,
                                                                     2000]},
                   scoring='roc_auc', verbose=1)

In [60]:
score = rs.score(X_train, y_train)
print(score)

0.9502933330592982


In [61]:
score = rs.score(X_test, y_test)
print(score)

0.8242654945054946


In [62]:
import joblib
joblib.dump(rs, 'rfc2.pkl')

['rfc2.pkl']

In [63]:
from sklearn.ensemble import GradientBoostingClassifier
pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('classifier',GradientBoostingClassifier())
           ])

In [64]:
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
parameters = {"classifier__learning_rate": sp_randFloat(),
                  "classifier__subsample"    : sp_randFloat(),
                  "classifier__n_estimators" : sp_randInt(100, 1000),
                  "classifier__max_depth"    : sp_randInt(4, 10)
                 }

In [65]:
rs = RandomizedSearchCV(pipeline, parameters, cv=10, verbose=1,n_jobs=-1,scoring='roc_auc')
rs.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('numeric',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                                              ('categorical',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='constant')),
                                                                                               ('enco...
                   param_distributions={'classifier__learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7224a1c520>,
                                        'classifier__max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7224a1c850>,
                                        'classifier__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7224a1cdf0>,
                                        'classifier__subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f7224a1ca60>},
                   scoring='roc_auc', verbose=1)

In [66]:
score = rs.score(X_train, y_train)
print(score)

0.9875244369750003


In [67]:
score = rs.score(X_test, y_test)
print(score)

0.8148676923076923


In [68]:
import joblib
joblib.dump(rs, 'gbc.pkl')

['gbc.pkl']

In [69]:
from sklearn.svm import SVC
pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('classifier',SVC())
           ])

In [70]:
from scipy import stats
rand_list = {"classifier__C": stats.uniform(2, 10),
             "classifier__gamma": stats.uniform(0.1, 1)}

In [71]:
rs = RandomizedSearchCV(pipeline, rand_list, cv=10, verbose=1,n_jobs=-1,scoring='roc_auc')
rs.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('numeric',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                                              ('categorical',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='constant')),
                                                                                               ('enco...
                                                                                               ('encoder',
                                                                                                OneHotEncoder())]),
                                                                               ['gender',
                                                                                'Partner',
                                                                                'Dependents',
                                                                                'PhoneService',
                                                                                'PaperlessBilling'])])),
                                             ('classifier', SVC())]),
                   n_jobs=-1,
                   param_distributions={'classifier__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f72249c5340>,
                                        'classifier__gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f72249bd8e0>},
                   scoring='roc_auc', verbose=1)

In [72]:
score = rs.score(X_train, y_train)
print(score)

0.964806623132307


In [73]:
score = rs.score(X_test, y_test)
print(score)

0.7598945054945054


## Conclusiones
**¿Se evidencian problemas de overfitting o underfitting?**

El SVC que entrenamos tiene serios problemas de overfitting, siendo su resultado en entrenamiento bastante mas alto que el de testing, mientras que los otros dos modelos se veian mucho mejor. En ambos modelos evidenciamos (Random Forest - Gradient Boosting) un comportamiento bastante mejor con respecto al overfitting, aunque si encontramos que el RandomizedSearch entreno el mejor modelo maximizando score, sin revisar el overfitting. Sin embargo, decidimos utilizar estos modelos ya que aun con este problema tiene un muy buen rendimiento con datos que no conoce. Los resultados del ROC AUC fueron los siguientes:


*   Random Forest: 95% en entrenamiento y 82% en testing. 
*   Gradient Boosting: 99% en entrenamiento y 81% en testing.
*   SVC: 96% en entrenamiento y 75% en testing.



Finalmente, decidimos escoger el Random Forest como nuestro modelo a desplegar pues presentaba un mejor rendimiento en promedio entre las diferentes clases. Ademas con datos desconocidos ambos modelos se comportan de manera muy similar y es el que menos overfitting presentaba.

**Curva ROC AUC vs. F1 score vs. Precision/Recall**

Escoger una buena metrica de evaluacion es algo muy importante ya que dependiendo del contexto y del objetivo que se tenga puede resultar mejor o peor esocger alguna de las metricas. Sin embargo, la metrica ROCA AUC tiene una ventaja sobre las demas metricas. Su ventaja principal es que puede funcionar con problemas donde tengamos datos desbalanceados. Contrario al F1-score donde se castiga mucho que las clases esten desbalanceadas. Por esto, en algunos problemas puede resultar mejor usar el ROC AUC, sobretodo en aquellos donde le damos la misma importancia a la clase positiva y negativa.

Fuente: https://neptune.ai/blog/f1-score-accuracy-roc-auc-pr-auc

